In [2]:
from newspaper import Article
import numpy as np  
import newspaper

## get urls and keywords from file

In [3]:
with open('urls.txt', 'r') as f:
    urls = f.readlines()
urls = [url.replace('\n', '') for url in urls]
urls

['https://vnexpress.net/', 'https://www.24h.com.vn/', 'https://tuoitre.vn/']

In [4]:
with open('keywords.txt', 'r') as f:
    keywords = f.readlines()
keywords = [key.replace('\n', '') for key in keywords]
keywords

['thủ tướng', 'chính phủ', 'Phạm Minh Chính']

### standardize keywords

In [5]:
keywords = [key.replace(' ', '_') for key in keywords]
keywords = [key.lower() for key in keywords]
list_key_tmp = []
for key in keywords:
    list_key_tmp += key.split('_')

# keywords += list_key_tmp
keywords

['thủ_tướng', 'chính_phủ', 'phạm_minh_chính']

## crawl function

In [6]:
from bs4 import BeautifulSoup
import requests
import re  

def is_valid(url):
    # return re.findall(r'\.[a-z]{3}', url)
    return re.findall(r'\.[a-z]{3}', url) and re.search(r'https://', url)

# global dict
dict_url = {}

def get_link_articles_from_url(url):
    respone = requests.get(url)
    soup = BeautifulSoup(respone.text, 'html.parser')
    
    for a in soup.find_all('a', href=True):
        articles_link = a['href'].replace('#box_comment_vne', '')
        articles_link = articles_link.replace('#box_comment', '')
       
        if articles_link not in dict_url.keys() and is_valid(articles_link):
            dict_url[articles_link] = 1

    return list(dict_url.keys())

# test 
tmp = get_link_articles_from_url(urls[1])
print(len(tmp))
print(*tmp[:5], sep='\n')


275
https://www.24h.com.vn/bong-da-c48.html
https://www.24h.com.vn/kinh-doanh-c161.html
https://www.24h.com.vn/thi-truong-tieu-dung-c52.html
https://www.24h.com.vn/suc-khoe-doi-song-c62.html
https://www.24h.com.vn/thoi-trang-hi-tech-c407.html


In [7]:
import newspaperedited

def get_category(url_home):
    dict_cate = {}
    home = newspaperedited.build(url_home)
    category_urls = home.category_urls()
    return list(set(category_urls))

list_cate = [get_category(url) for url in urls]

In [8]:
list_categorys = []
for i in list_cate:
    list_categorys += i  
list_categorys += urls
list_categorys

['https://vnexpress.net/so-hoa',
 'https://vnexpress.net/goc-nhin',
 'https://vnexpress.net/du-lich',
 'https://vnexpress.net/suc-khoe',
 'https://vnexpress.net/phap-luat',
 'https://vnexpress.net/oto-xe-may',
 'https://vnexpress.net',
 'https://vnexpress.net/giao-duc',
 'https://vnexpress.net/kinh-doanh',
 'https://vnexpress.net/thoi-su',
 'https://e.vnexpress.net',
 'https://vnexpress.net/khoa-hoc',
 'https://vnexpress.net/',
 'https://vnexpress.net/doi-song',
 'https://vnexpress.net/giai-tri',
 'https://vnexpress.net/tin-tuc-24h',
 'https://vnexpress.net/the-thao',
 'https://vnexpress.net/y-kien',
 'https://video.vnexpress.net',
 'https://vnexpress.net/hai',
 'https://vnexpress.net/tam-su',
 'https://vnexpress.net/the-gioi',
 'https://www.24h.com.vn',
 'https://www.24h.com.vn/',
 'http://baogia.24h.com.vn',
 'https://tuoitre.vn/giai-tri.htm',
 'https://beta.tuoitre.vn',
 'https://tuoitre.vn/can-biet.htm',
 'https://tuoitre.vn/xe.htm',
 'https://tuoitre.vn/',
 'https://tuoitre.vn/suc

## get keywords from url

In [9]:
def get_text_url(url):
    text = ''
    try:
        article = Article(url)
        article.download()
        article.parse()
    except:
        return text, url
    text = article.text.replace('\n', '.\n')
    url = article.url
    return text, url


In [10]:
from pyvi import ViTokenizer
from collections import Counter

def get_keywords_from_text(text):
    tokens = ViTokenizer.tokenize(text)
    tokens = ViTokenizer.spacy_tokenize(tokens)[0]
    tokens = list(filter(lambda x: len(x)>1, tokens))
    counter_tokens = Counter(tokens)
    counter_tokens = dict(counter_tokens)
    counter_tokens = dict(sorted(counter_tokens.items(), key=lambda x:-x[1]))
    return counter_tokens


In [11]:
def get_important_score(dict_keywords, keys):
    score = 0
    for key in dict_keywords.keys():
        for k in keys:
            if k == key.lower():
                score += dict_keywords[key]
    return score


## ranking articles

In [12]:
list_article_url = []

dict_url = {}
for category in list_categorys:
    list_article_url = get_link_articles_from_url(category)

list_article_url = list(set(list_article_url))

In [13]:
len(list_article_url)

1218

In [14]:
# t = get_text_url(list_article_url[100])
# print(list_article_url[100])
# print(t)

In [15]:
list_text_url = []
cnt = 0
for article_url in list_article_url:
    text, url = get_text_url(article_url)
    if text != '':
        list_text_url.append((text, url))
    else:
        continue

    if cnt%10==0:
        print(len(list_text_url), end='-')
    cnt += 1

1-11-21-31-41-51-61-71-81-91-101-111-121-131-141-151-161-171-181-191-201-211-221-231-241-251-261-271-281-291-301-311-321-331-341-351-361-371-381-391-401-411-421-431-441-451-461-471-481-491-501-511-521-531-541-551-561-571-581-591-601-611-621-631-641-651-661-671-681-691-701-711-721-731-741-751-761-771-781-791-801-811-821-831-841-851-861-871-881-891-901-911-921-931-941-951-961-971-981-991-1001-1011-1021-1031-1041-1051-1061-1071-1081-1091-1101-1111-1121-1131-1141-1151-

In [16]:
print(len(list_text_url))

1154


In [17]:
dict_index_and_score = {i:0 for i in range(len(list_text_url))}
# dict_index_and_score {0:0, 1:0, 2:0, .....}

In [18]:
list_keys = []
cnt = 0
for text, url in list_text_url:
    keys = get_keywords_from_text(text)
    dict_index_and_score[cnt] = get_important_score(keys, keywords)
    if cnt%10==0:
        print(cnt, end='-')
    cnt += 1


0-10-20-30-40-50-60-70-80-90-100-110-120-130-140-150-160-170-180-190-200-210-220-230-240-250-260-270-280-290-300-310-320-330-340-350-360-370-380-390-400-410-420-430-440-450-460-470-480-490-500-510-520-530-540-550-560-570-580-590-600-610-620-630-640-650-660-670-680-690-700-710-720-730-740-750-760-770-780-790-800-810-820-830-840-850-860-870-880-890-900-910-920-930-940-950-960-970-980-990-1000-1010-1020-1030-1040-1050-1060-1070-1080-1090-1100-1110-1120-1130-1140-1150-

In [19]:
list_index = dict(sorted(dict_index_and_score.items(), key=lambda x : -x[1]))

In [23]:
cnt = 0 
ans = []
for i in list_index:
    cnt+=1 
    print(i, list_index[i])
    ans.append(list_text_url[i][1])
    print(list_text_url[i][1])
    if cnt == 20:
        break

362 25
https://vnexpress.net/chinh-phu-moi-nen-uu-tien-nhung-gi-4257968.html
1071 14
https://vnexpress.net/chuyen-gia-duc-danh-gia-cao-thu-tuong-pham-minh-chinh-4258753.html
405 11
https://vnexpress.net/lan-song-dich-thu-hai-4100535.html
567 10
https://vnexpress.net/trinh-mien-nhiem-13-thanh-vien-chinh-phu-4259027.html
827 9
https://vnexpress.net/chinh-phu-moi-4256811.html
595 8
https://www.24h.com.vn/tin-tuc-trong-ngay/trinh-mien-nhiem-mot-so-pho-thu-tuong-va-bo-truong-c46a1241174.html
377 6
https://vnexpress.net/phap-bom-them-4-ty-euro-cuu-air-france-4258920.html
928 5
https://vnexpress.net/doi-ngoai-viet-nam-duoc-danh-gia-no-ro-5-nam-qua-4257993.html
1060 5
https://vnexpress.net/anh-chuan-bi-trien-khai-ho-chieu-vaccine-4258142.html
1137 5
https://vnexpress.net/chinh-phu-huu-hieu-4257910.html
123 4
https://vnexpress.net/singapore-ung-dung-ky-thuat-so-de-som-khoi-phuc-du-lich-4255530.html
270 4
https://vnexpress.net/campuchia-co-the-cho-benh-nhan-covid-19-dieu-tri-tai-nha-4259014.html

## export data

In [25]:
path = 'datacrawled/'
cnt = 0
dict_title = {}
for url in ans:

    article = Article(url)
    article.download()
    article.parse()
    article.nlp()
    text = article.text.replace('\n', '.\n')
    title = article.title
    summary = article.summary
    url = article.url
    if title not in dict_title.keys():
        dict_title[title] = 1
        cnt += 1
        with open(path+'article'+str(cnt)+'.txt', 'w') as f:
            f.write(title+'\n')
            f.write(url+'\n')
            f.write(summary+'\n')